### Q1. Getting the embeddings model


In [3]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

/home/codespace/.python/current/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
user_question = "I just discovered the course. Can I still join it?"
embedding_model.encode(user_question)[0]


0.078222655

### Prepare the documents

In [5]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

### Q2. Creating the embeddings


In [6]:
import numpy as np
embeddings = []
for doc in documents:
    qa_text = f"{doc['question']} {doc['text']}"
    embeddings.append(embedding_model.encode(qa_text))

In [7]:
X = np.array(embeddings)
X.shape

(948, 768)

### Q3. Search

In [8]:
v = embedding_model.encode(user_question)
scores = X.dot(v)
max(scores)

0.6506573

### Vector search

In [9]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'text': 'Star the r

### Q4. Hit-rate for our search engine

In [12]:
import pandas as pd
from tqdm.auto import tqdm


base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [11]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)


def evaluate(ground_truth, search_function, embedding_model):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        v = embedding_model.encode(q['question'])
        results = search_function(v)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [13]:
evaluate(ground_truth, search_engine.search, embedding_model)


100%|██████████| 1830/1830 [02:17<00:00, 13.27it/s]


{'hit_rate': 0.9431693989071038, 'mrr': 0.8231173562321104}

### Q5. Indexing with Elasticsearch

In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
# indexing documents, embeddings are computed already!
for i,doc in tqdm(enumerate(filtered_documents)):
    doc['question_text_vector'] = embeddings[i]
    es_client.index(index=index_name, document=doc)

In [ ]:
es_client.count(index=index_name)


In [ ]:
def elastic_search_knn_small(query_vector):
    search_query = {
        "knn": {
            "field": 'question_text_vector',
            "query_vector": query_vector,
            "k": 5,
            "num_candidates": 10000,
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
search_query = {
    "knn": {
        "field": 'question_text_vector',
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 10000,
    },
    "_source": ["text", "section", "question", "course", "id"]
}

In [ ]:
es_results = es_client.search(
        index=index_name,
        body=search_query
    )

In [ ]:
es_results['hits']['hits'][0]['_score']


In [ ]:
max([hit['_score'] for hit in es_results['hits']['hits']])


In [ ]:
es_results['hits']['hits'][0]['_source']['id']


### Q6. Hit-rate for Elasticsearch


In [ ]:
evaluate(ground_truth, search_engine.search, embedding_model)
